# Installing required packages

In [53]:
!pip3 install nltk
!pip3 install sklearn
!pip3 install numpy
!pip3 install matplotlib

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user install

## Downloading nltk packages

In [ ]:
import nltk
nltk.download('wordnet', 'stopwords', 'punkt')

# Importing packages

In [54]:
import json
from math import sqrt, log
import numpy as np
from nltk.stem import PorterStemmer, WordNetLemmatizer
from math import log
from nltk.corpus import stopwords as sw
import time

In [55]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Reading data

In [63]:
def get_data(path):    
    '''Reading the json files from passed path'''
    with open(path) as d:
        json_data = json.load(d)
    return json_data

# Data Operations

In [64]:
def get_doc_terms(doc_info):
    '''concatenate all terms (title + body) of the passed doc'''
    body_terms = doc_info['body'].split()
    title_terms = doc_info['title'].split()
    all_terms = body_terms + title_terms
    return all_terms

## Count term frequency over collection

### get_document_frequency:

Counts each term frequency and assign an index to the term. 

map_to_index structure is like the following: {term: index}

frequencies returns the frequency of each term: {term: frequency}

In [65]:
stopwords = sw.words('english')

In [66]:
def get_document_frequency(json_data, stemming=False, lemmatizing=False):
    vectors = {}
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    for doc_id, doc in enumerate(json_data):
        terms = get_doc_terms(doc)
        for term in terms:
            if term not in stopwords:
                if lemmatizing:
                    term = lemmatizer.lemmatize(term)
                if stemming:
                    term = stemmer.stem(term)
                if term not in vectors:
                    vectors[term] = []
                if doc_id not in vectors[term]:
                    vectors[term].append(doc_id)
    
    vectors_list = list(vectors.items())
    term_map_index = {}
    
    for i in range(len(vectors_list)):
        term = vectors_list[i][0]
        term_map_index[term] = i
    
    frequencies = []
    for word, freq_list in vectors_list:
        frequencies.append(len(freq_list))
    
    return term_map_index, frequencies

###  get_vector:

creates a matrix with two axis: 

1. category 
2. term

each element represents the idf * tf of term in the category.

In [67]:
def get_vector(json_data, term_map_index, frequencies, train_docs_no, stemming=False, lemmatizing=False):
    term_no = len(term_map_index)
    data_no = len(json_data)
    vector = np.zeros((data_no, term_no))
        
    for doc_id, doc in enumerate(json_data):
        terms = get_doc_terms(doc)
        doc_term_freq = {}
        
        for term in terms:
            if term not in stopwords:
                if lemmatizing:
                    term = lemmatizer.lemmatize(term)
                if stemming:
                    term = stemmer.stem(term)
                if term not in doc_term_freq:
                    doc_term_freq[term] = 0
                doc_term_freq[term] += 1
                
        for term in doc_term_freq:
            if term in term_map_index:
                index = term_map_index[term]
                doc_frequency = frequencies[index]
                tf = doc_term_freq[term]
                idf = log(train_docs_no/doc_frequency)
                vector[doc_id, index] = tf * idf
    
    return vector

### get_category

returns sequential category of all documents

In [68]:
def get_category(data):
    '''return list of categories for train and test docs'''
    categories = []
    
    for doc in data:
        category = doc['category']
        categories.append(category)
        
    return categories

## Assigning variables

Since the training data size is very large (about 13 GB of memory is needed), it has been reduced to the tweleveth. 

The same scenario is for the test data size. 

In [69]:
start_time = time.time()

train_data = get_data('./data/train.json')
test_data = get_data('./data/validation.json')
# print(list(train_data)[:10])


N = len(train_data)

N = N // 12

#randomly pick N data from train
train_data = train_data[:N]
test_data = test_data[:300]

term_map_index, frequencies = get_document_frequency(train_data, lemmatizing=True)

train_categories = get_category(train_data)
test_categories = get_category(test_data)
# print(train_categories)[:10]

#train and test vectors 
train_vector = get_vector(train_data, term_map_index, frequencies, N)
test_vector = get_vector(test_data, term_map_index, frequencies, N)

finish_time = time.time()
print('elapsed time:', finish_time - start_time)

elapsed time: 0.6932623386383057


### lemmatizing vs stemming:

As the result shows, the lemmatizing works better than stemming on the data.

# kNN (k Nearest Neighbor)

There are two methods used for measuring nearest neighbors: 
1. Cosine Similarity
2. Euclidean Distance

The Following sections would be the implementation of each method. 

## Cosine Similarity

Cosine Similarity = train.test^t / |train|.|test^t|

Since |test^t| is constant, we can ignore its value and don't calculate it. 

## Euclidean Distance

In [70]:
euclidean_distance = None
def calculate_euclidean_distance(train, test):
    '''Euclidean Distance'''
    global euclidean_distance
    x_2 = np.sum(test * test, axis=1)
    test_transpose = test.T
    xy = np.dot(train, test_transpose).T
    y_2 = np.sum(train * train, axis = 1)
    
    #make the test 1-column for evaluation
    x_2 = x_2[:,None]
    result = x_2 + y_2 - 2 * xy
    euclidean_distance = np.sqrt(result)
    return euclidean_distance


In [71]:
cosine_similarity = None
def calculate_cosine_similarity(train, test):
    '''consine_similarity = train.test^t / |train|.|test|'''
    global cosine_similarity
    test_transpose = test.T
    dot_product = np.dot(train, test_transpose).T
    normalization = np.linalg.norm(train, axis=1)
    cosine_similarity = dot_product / normalization
    return cosine_similarity


## kNN 

As the k becomes larger (in this case 5 has the best result among 1 and 3), the algorithm performs better. That sounds reasonable, because of more neighbors are checked to determine the class/category.

In [72]:
def knn(doc_id, method, k=5):
    global cosine_similarity
    global euclidean_distance
    if method == 'cosine':
        similarities = cosine_similarity[doc_id]
        nearest = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:k]
    elif method == 'euclidean':
        distances = euclidean_distance[doc_id]
        nearest = sorted(range(len(distances)), key=lambda i: distances[i])[:k]
    categories = {}
    for i in range(len(nearest)):
        category = train_categories[nearest[i]]
        if category not in categories:
          categories[category] = 0
        categories[category] += 1
    
    return max(categories, key=categories.get)
    


# Performance Measure

In [73]:
def get_vector_creation_time(method):
    start_time = time.time()
    
    if method == 'euclidean':
        calculate_euclidean_distance(train_vector, test_vector)
    elif method == 'cosine':
        calculate_cosine_similarity(train_vector, test_vector)
    finish_time = time.time()
    elapsed_time = finish_time - start_time
    return elapsed_time

In [74]:
def get_recall(method, category_no=4, k=5):
    tp, total = [], []
    
    for i in range(category_no):
        tp.append(0)
        total.append(0)
        
    test_no = len(test_data)
    for i in range(test_no):
        real = test_data[i]['category']
        if method == 'bayes':
          predicted = naive_bayes_testing(test_data[i], 1)
        else:
          predicted = knn(i, method, k)
        index = real - 1
        
        if real == predicted:
            tp[index] += 1
            
        total[index] += 1
    
    recall = {}
    
    for i in range(category_no):
        index = i + 1
        recall[index] = tp[i]/total[i]
    
    return recall

In [75]:
def get_precision(method, category_no=4, k=5):
    tp, total = [], []
    
    for i in range(category_no):
        tp.append(0)
        total.append(0)
        
    test_no = len(test_data)
    for i in range(test_no):
        real = test_data[i]['category']
        if method == 'bayes':
          predicted = naive_bayes_testing(test_data[i], 1)
        else:
          predicted = knn(i, method, k)
        index = predicted - 1
        
        if real == predicted:
            tp[real - 1] += 1
            
        total[index] += 1
    
    precision = {}
    
    for i in range(category_no):
        index = i + 1
        precision[index] = tp[i]/total[i]
    
    return precision

In [76]:
def get_accuracy(method, category_no=4, k=5):
    test_no = len(test_data)
    tp = 0
    
    for i in range(test_no):
        if method == 'bayes':
          predicted = naive_bayes_testing(test_data[i], 1)
        else:
          predicted = knn(i, method, k)
        real = test_data[i]['category']
        
        if predicted == real:
            tp += 1
    
    accuracy = tp/test_no
    return accuracy

In [77]:
def get_marco_f1(precision_val, recall_val, category_no=4):
    results = []
    for i in range(1, category_no+1):
        result = 2 * precision_val[i] * recall_val[i] / (precision_val[i] + recall_val[i])
        results.append(result)
    return np.mean(results)

## Euclidean

In [78]:
get_vector_creation_time('euclidean')

0.7487761974334717

In [79]:
euclidean_recall = get_recall('euclidean')
euclidean_recall

{1: 0.36046511627906974,
 2: 0.04054054054054054,
 3: 0.9382716049382716,
 4: 0.2542372881355932}

In [80]:
euclidean_precision = get_precision('euclidean')
euclidean_precision

{1: 0.7560975609756098, 2: 1.0, 3: 0.3247863247863248, 4: 0.6818181818181818}

In [81]:
get_accuracy('euclidean')

0.4166666666666667

In [82]:
get_marco_f1(euclidean_precision, euclidean_recall)

0.35475527680252095

In [83]:
get_vector_creation_time('cosine')

0.7680037021636963

In [84]:
cosine_recall = get_recall('cosine')
cosine_recall

{1: 0.8023255813953488,
 2: 0.8918918918918919,
 3: 0.7530864197530864,
 4: 0.7796610169491526}

In [85]:
cosine_precision = get_precision('cosine')
cosine_precision

{1: 0.8846153846153846,
 2: 0.868421052631579,
 3: 0.782051282051282,
 4: 0.6764705882352942}

In [86]:
get_accuracy('cosine')

0.8066666666666666

In [87]:
get_marco_f1(cosine_precision, cosine_recall)

0.8032921152343301

# Naive Bayes

For testing naive bayes with a constant value of alpha, the best result was for 0.368 for 10 randomly generated values between 0 and 1. 

In [88]:
n_c, t_c = None, None
lemmatizer = WordNetLemmatizer()
def naive_bayes_training(data, category_no=4):
    global n_c
    global t_c
    term_no = len(term_map_index)
    
    n_c = []
    for i in range(category_no):
        n_c.append(0)
    
    t_c = np.zeros((category_no, term_no))
    
    for doc_id, doc in enumerate(data):
        terms = get_doc_terms(doc)
        category = doc['category']

        for term in terms:
            index = category - 1
            if term not in stopwords:
                term = lemmatizer.lemmatize(term)
                t_c[index, term_map_index[term]] += 1
            n_c[index] += 1
            

In [100]:
def naive_bayes_testing(document, alpha=0.368, category_no=4):
    global n_c
    global t_c
    term_no = len(term_map_index)
    scores = []
    
    for i in range(category_no):
        score = log(n_c[i]/N)
        scores.append(score)
    
    sigma = np.sum(t_c, axis=1)
    
    for i in range(category_no):
        terms = get_doc_terms(document)
        for term in terms:
            if term not in stopwords:
                term = lemmatizer.lemmatize(term)
                if term in term_map_index:
                    fraction = log((t_c[i, term_map_index[term]] + alpha) / (sigma[i] + term_no))
                    scores[i] += fraction
    predicted = np.argmax(scores) + 1

    return predicted


## Performance Measure

In [90]:
naive_bayes_training(train_data)

In [91]:
bayes_recall = get_recall('bayes')
bayes_recall

{1: 0.9069767441860465,
 2: 0.9459459459459459,
 3: 0.8641975308641975,
 4: 0.864406779661017}

In [92]:
bayes_precision = get_precision('bayes')
bayes_precision

{1: 0.9512195121951219,
 2: 0.9210526315789473,
 3: 0.9090909090909091,
 4: 0.7846153846153846}

In [93]:
get_accuracy('bayes')

0.8966666666666666

In [94]:
get_marco_f1(bayes_precision, bayes_recall)

0.8926403391082852

# Random Forest

Changing the default value of C (1), doesn't show noticable change in the results

In [95]:
start_time = time.time()
clf = svm.SVC(kernel='linear', C=1)
clf.fit(train_vector, train_categories)
finish_time = time.time()
elapsed_time = finish_time - start_time

In [96]:
elapsed_time

143.78522396087646

In [97]:
clf.score(test_vector, test_categories)

0.8433333333333334

In [98]:
start_RandomForestClassifiermForestClassifiermax_depth=Classifier= time.time()
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(train_vector, train_categories)
finish_time = time.time()
elapsed_time = finish_time - start_time

In [99]:
clf.score(test_vector, test_categories)

0.6733333333333333